<h1> W207 Final </h1>
<h3> Group Members: </h3>
Ben Attix <br>
Justin Plumley <br>
Dan Watson <br>
<h3>Kaggle Competition: <br> Personalized Medicine: Redefining Cancer Treatment </h3><br>
https://www.kaggle.com/c/msk-redefining-cancer-treatment<br>
<h5> Evaluation </h5>
Submissions are evaluated on Multi Class Log Loss between the predicted probability and the observed target.

In [1]:
# General libraries.
import numpy as np
from numpy import genfromtxt
import matplotlib.pyplot as plt
import pandas as pd

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [21]:
#Load data- easiest into dataframe then convert to np.array
train_variants_df = pd.read_csv("/Users/Dan/Documents/Berkeley/w207_Kaggle/training_variants")
test_variants_df = pd.read_csv("/Users/Dan/Documents/Berkeley/w207_Kaggle/test_variants")
train_text_df = pd.read_csv("/Users/Dan/Documents/Berkeley/w207_Kaggle/training_text", sep="\|\|", engine="python", skiprows=1, names=["ID", "Text"])
test_text_df = pd.read_csv("/Users/Dan/Documents/Berkeley/w207_Kaggle/test_text", sep="\|\|", engine="python", skiprows=1, names=["ID", "Text"])

# convert dfs into np arrays
train_variants = train_variants_df.as_matrix()
test_variants = test_variants_df.as_matrix()
train_text = train_text_df.as_matrix()
test_text = test_text_df.as_matrix()

print("Train Variants shape: " + str(train_variants.shape))
print("Test Variants shape: " + str(test_variants.shape))
print("Train Text shape: " + str(train_text.shape))
print("Test Text shape: " + str(test_text.shape))

Train Variants shape: (3321, 4)
Test Variants shape: (5668, 3)
Train Text shape: (3321, 2)
Test Text shape: (5668, 2)


In [27]:
## View dataframes 
train_variants_df.head()
#test_variants_df.head()
#train_text_df.head()
#train_text_df.head()


,ID,Gene,Variation,Class
0,0,FAM58A,Truncating Mutations,1
1,1,CBL,W802*,2
2,2,CBL,Q249E,2
3,3,CBL,N454D,3
4,4,CBL,L399V,4


In [29]:
y_train_variants, X_train_variants = train_variants [:, 3:], train_variants[:, :3]

In [32]:
X_train_variants

array([[0, 'FAM58A', 'Truncating Mutations'],
       [1, 'CBL', 'W802*'],
       [2, 'CBL', 'Q249E'],
       ..., 
       [3318, 'RUNX1', 'Fusions'],
       [3319, 'RUNX1', 'R80C'],
       [3320, 'RUNX1', 'K83E']], dtype=object)

In [34]:
#Simple output for initial submission- blatantly taken from Kaggle
class_probs = []
for i in range(1,10):
    class_probs.append(train_variants_df[train_variants_df['Class'] == i].shape[0]/3321.)
    
submission = pd.read_csv("/Users/Dan/Documents/Berkeley/w207_Kaggle/submissionFile")

for i in range(9):
    submission['class'+str(i+1)] = class_probs[i]
    
submission.to_csv("dumb_submission.csv", index=False)
submission.head()

,ID,class1,class2,class3,class4,class5,class6,class7,class8,class9
0,0,0.171033,0.136104,0.026799,0.206564,0.07287,0.082806,0.286962,0.005721,0.011141
1,1,0.171033,0.136104,0.026799,0.206564,0.07287,0.082806,0.286962,0.005721,0.011141
2,2,0.171033,0.136104,0.026799,0.206564,0.07287,0.082806,0.286962,0.005721,0.011141
3,3,0.171033,0.136104,0.026799,0.206564,0.07287,0.082806,0.286962,0.005721,0.011141
4,4,0.171033,0.136104,0.026799,0.206564,0.07287,0.082806,0.286962,0.005721,0.011141
